In [ ]:
import sys
sys.path.append('../input/iterativestratificationkfold/iterative-stratification-master/iterative-stratification-master')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
test_features = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
train_targets = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
train_features = pd.read_csv("/kaggle/input/lish-moa/train_features.csv")
submission = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import tensorflow as tf
import io
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [ ]:
#pip install iterative-stratification

In [ ]:
#preprocessing steps
def preprocess(df):
    df.loc[:, 'cp_type'] = df.loc[:, 'cp_type'].map({'trt_cp': 0, 'ctl_vehicle': 1})
    df.loc[:, 'cp_dose'] = df.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})
    df.loc[:, 'cp_time'] = df.loc[:, 'cp_time'].map({24: 0, 48: 1, 72: 2})
    del df['sig_id']
    return df

In [ ]:
train = preprocess(train_features)
test = preprocess(test_features)

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
del train_targets['sig_id']

In [ ]:
print(train.shape)
print(train_targets.shape)

In [ ]:
train = train.astype('float32')
test = test.astype('float32')
train_targets = train_targets.astype('float32')

In [ ]:
def metric(y_true, y_pred):
    metrics = []
    for _target in train_targets.columns:
        metrics.append(log_loss(y_true.loc[:, _target], y_pred.loc[:, _target].astype(float),labels=[0,1]))
    return np.mean(metrics)

In [ ]:
import keras
import tensorflow
from keras.layers import Input, concatenate, Activation, Add
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.utils import to_categorical
from numpy.random import seed
from keras.layers import Conv1D
from keras.layers import MaxPooling1D, GlobalAveragePooling1D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.layers import Dropout
from tensorflow.keras import regularizers
from keras.models import Sequential
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import log_loss
import math
from keras.callbacks import LearningRateScheduler
from keras.layers import LeakyReLU

seed(1)
tf.random.set_seed(1)

In [ ]:
from tensorflow.keras import regularizers
def evaluate_model(train): 
        input_shape = train.shape[1]
        input = Input(shape=(input_shape,1))
        input_shortcut = input
        convolved = Conv1D(64, 2, padding="same",kernel_regularizer=keras.regularizers.l2(l=0.1))(input)
        convolved= LeakyReLU(alpha=0.1)(convolved)
        x_1= MaxPooling1D()(convolved)
        convolved_4= Conv1D(64, 8, padding="same",kernel_regularizer=keras.regularizers.l2(l=0.1))(input)
        convolved_4= LeakyReLU(alpha=0.1)(convolved_4)
        x_15= MaxPooling1D()(convolved_4)
        convolved1 = Conv1D(64, 16, padding="same",kernel_regularizer=keras.regularizers.l2(l=0.1))(input)
        convolved1= LeakyReLU(alpha=0.1)(convolved1)
        x_2= MaxPooling1D()(convolved1)
        c = concatenate([x_1, x_2,x_15])
        convolved4 = Conv1D(64, 3, padding="same",kernel_regularizer=keras.regularizers.l2(l=0.1))(c)
        convolved4= LeakyReLU(alpha=0.1)(convolved4)
        x_4= MaxPooling1D()(convolved4)    
        convolved5 = Conv1D(64, 3, padding="same",kernel_regularizer=keras.regularizers.l2(l=0.1))(x_4)
        convolved5= LeakyReLU(alpha=0.1)(convolved5)
        x_5= MaxPooling1D()(convolved5)
        x_5 = Dropout(0.50)(x_5)
        x_6 = Flatten()(x_5)
        x_6 = Dropout(0.50)(x_6)
        x_7 = Dense(256)(x_6)
        x_10 = Flatten()(input_shortcut)
        x_10 = Dense(256)(x_10)
        x_9 = Add()([x_10,x_7])
        x_9 = Activation('linear')(x_9)
        x_9 = Dropout(0.50)(x_9)
        output = Dense(units=206, activation='sigmoid')(x_9)
        model3 = Model(input,output)
        opt = keras.optimizers.Adam(learning_rate=0.001)
        model3.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
        return model3

In [ ]:
N_STARTS = 7
res = train_targets.copy()
res.loc[:, train_targets.columns] = 0
submission.loc[:, train_targets.columns] = 0
historys = dict()
tf.random.set_seed(43)
for seed in range(N_STARTS):
    for n, (tr, te) in enumerate(MultilabelStratifiedKFold(n_splits=7, random_state=seed, shuffle=True).split(train, train_targets)):
        print(f"======{train.iloc[tr].shape}========{train_targets.iloc[tr].shape}=====")
        print(f'Seed: {seed} => Fold: {n}')
        model = evaluate_model(train)
        checkpoint_path = f'repeat:{seed}_Fold:{n}.hdf5'
        reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, min_lr=1e-5, patience=3, verbose=1, mode='min')
        cb_checkpt = ModelCheckpoint(checkpoint_path, monitor = 'val_loss', verbose = 1, save_best_only = True,
                                     save_weights_only = True, mode = 'min')
        early = EarlyStopping(monitor="val_loss", mode="min", restore_best_weights=True, patience= 5, verbose = 1)
        history = model.fit(train.iloc[tr], train_targets.iloc[tr],
                  validation_data=(train.values[te], train_targets.iloc[te]),
                  epochs=50, batch_size=128,
                  callbacks=[reduce_lr_loss, cb_checkpt, early], verbose=2
                 )
        historys[f'history_{seed+1}'] = history
        print("Model History Saved.")
        model.load_weights(checkpoint_path)
        #test_predict = model.predict(test.values[:, top_feats])
        test_predict = model.predict(test)
        val_predict = model.predict(train.iloc[te])
        res.loc[te, train_targets.columns] += val_predict
        submission.loc[:, train_targets.columns] += test_predict

        print(f'OOF Metric For FOLD {n} : {metric(train_targets.loc[te, train_targets.columns], pd.DataFrame(val_predict, columns=train_targets.columns))}')
        print('+-' * 10)
res.loc[:, train_targets.columns] /= N_STARTS
    
submission.loc[:, train_targets.columns] /= ((n+1) * N_STARTS)

In [ ]:

print(f'OOF Metric: {metric(train_targets, res)}')


In [ ]:
submission.to_csv('submission.csv', index=False)